In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from transformers.utils import is_torch_available, is_tf_available
import pandas as pd

In [ ]:
df = pd.read_csv('INPUT/emociones.csv')
df = df[['opinion', 'label']]
df = df.dropna()
df = df.sample(30, random_state=42).reset_index(drop=True)
print(df)

In [ ]:
zero_shot_prompt = """Clasifica el sentimiento del siguiente texto como positivo, negativo o neutral:

"{}"
"""

few_shot_prompt = """Clasifica el sentimiento del siguiente texto como positivo, negativo o neutral.
Ejemplo 1: \"Me encantó la película\" → positivo
Ejemplo 2: \"No me gustó para nada\" → negativo
Ejemplo 3: \"Está bien, sin más\" → neutral
Texto: "{}"
Respuesta:"""

cot_prompt = """Analiza el siguiente texto y razona si es positivo, negativo o neutral.
Texto: "{}"
Primero, explica por qué. Luego, indica si es positivo, negativo o neutral."""

prompts = {
    "zero-shot": zero_shot_prompt,
    "few-shot": few_shot_prompt,
    "chain-of-thought": cot_prompt
}

In [ ]:
def predict_qwen(tokenizer, ):
    prompt = prompt_template.format(opinion)
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    prompt = "Give me a short introduction to large language model."
    messages = [
        {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response.choices[0].message["content"].strip().lower()

model_name = "Qwen/Qwen2.5-1.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

results = {}

for method, prompt_template in prompts.items():
    predictions = []
    for opinion in df['opinion']:
        pred = predict_openai(opinion,prompt_template)
        pred = pred.lower()
        if "positivo" in pred:
            predictions.append("positiva")
        elif "negativo" in pred:
            predictions.append("negativa")
        elif "neutral" in pred:
            predictions.append("neutral")
        else:
            predictions.append("desconocido")  # Fallback
    results[method] = predictions

for method, preds in results.items():
    print(f"\n=== {method.upper()} ===")
    print(classification_report(df['label'], preds, zero_division=0))